In [10]:
# ============================================================================
# Chronos T5 Base - Google Colab Portfolio Backtest
# ============================================================================

from google.colab import drive
import sys
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import yfinance as yf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import f as f_dist
import statsmodels.api as sm
from datetime import datetime
import matplotlib.dates as mdates
import joblib

drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/chronos_t5_base_project_portfolio
%cd /content/drive/MyDrive/chronos_t5_base_project_portfolio

!git clone https://github.com/amazon-science/chronos-forecasting
%cd chronos-forecasting

%pip install torch transformers datasets accelerate scikit-learn tqdm joblib

sys.path.append("src")
from chronos import BaseChronosPipeline

def set_seed(seed=42):
    """Set all random seeds for reproducibility (CUDA version)"""
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Total memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/chronos_t5_base_project_portfolio
fatal: destination path 'chronos-forecasting' already exists and is not an empty directory.
/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos-forecasting
Using device: cuda
Device name: Tesla T4
Total memory: 14.7 GB


In [11]:
# Load Chronos T5-Base pretrained model
try:
    pipeline = BaseChronosPipeline.from_pretrained(
        "amazon/chronos-t5-base",
        device_map="auto" if torch.cuda.is_available() else "cpu",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32  # Use FP16 to save memory
    )
    print(f"Successfully loaded Chronos T5 Base model on {device}")

    if hasattr(pipeline.model, 'device'):
        print(f"Model device: {pipeline.model.device}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("CUDA cache cleared")

except Exception as e:
    print(f"Failed to load model: {e}")
    raise

Successfully loaded Chronos T5 Base model on cuda
Model device: cuda:0
CUDA cache cleared


In [12]:
data_path = "/content/drive/MyDrive/ERP Data/all_window_datasets_unscaled.npz"

def load_datasets(data_path):
    """Load dataset"""
    try:
        data = np.load(data_path, allow_pickle=True)
        print(f"Data loaded successfully! Keys: {list(data.keys())}")

        datasets = {}
        for key in data.files:
            datasets[key] = data[key]
        print(f"Available datasets: {len(datasets)} files")
        return datasets

    except FileNotFoundError:
        print(f"Data file not found: {data_path}")
        print("Please ensure all_window_datasets.npz is uploaded to Google Drive 'ERP Data' folder")
        raise
    except Exception as e:
        print(f"Error loading data: {e}")
        raise

datasets = load_datasets(data_path)

WINDOW_SIZES = [5, 21, 252, 512]
START_YEAR = 2016
END_YEAR = 2024
TRANSACTION_COST = 0.0015

def get_batch_size(window_size):
    """Return suitable batch size for GPU based on window size"""
    if window_size <= 5:
        return 1024
    elif window_size <= 21:
        return 256
    elif window_size <= 252:
        return 32
    elif window_size <= 512:
        return 16
    else:
        return 8

print("GPU batch size configuration (T5-Base):")
for ws in WINDOW_SIZES:
    batch_size = get_batch_size(ws)
    print(f"  Window {ws}: Batch size = {batch_size}")

print("Configuration complete!")

In [13]:
def annual_sharpe(rets, freq=252):
    mu = float(np.mean(rets)) * freq
    sd = float(np.std(rets, ddof=1)) * np.sqrt(freq)
    return mu / sd if sd > 0 else 0

# Load risk-free rate and calculate S&P500 Excess Sharpe

rf_file = "/content/drive/MyDrive/ERP Data/CRSP_2016_2024_top50_with_exret.csv"
try:
    rf_df = pd.read_csv(rf_file, usecols=["date", "rf"])
    rf_df["date"] = pd.to_datetime(rf_df["date"])
    rf_df = rf_df.drop_duplicates("date").set_index("date").sort_index()
    rf_series = rf_df["rf"].astype(float)

    px = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")["Close"]
    sp_ret = px.pct_change().dropna()
    rf_align = rf_series.reindex(sp_ret.index).fillna(method="ffill")
    sp_excess = sp_ret.values - rf_align.values

    SR_MKT_EX = annual_sharpe(sp_excess)
    print(f"[INFO] S&P500 Excess Sharpe (2016–24) = {SR_MKT_EX:.3f}")
except Exception as e:
    print(f"Warning: Could not load risk-free rate data: {e}")
    SR_MKT_EX = 0.5

def delta_sharpe(r2_zero: float, sr_base: float):
    """
    If r2_zero <= 0   -> ΔSharpe = 0, Sharpe* = sr_base
    If r2_zero >= 1   -> ΔSharpe = 0, Sharpe* = sr_base (extreme case fallback)
    Otherwise, use the original formula
    """
    if (r2_zero <= 0) or (r2_zero >= 1):
        return 0.0, sr_base
    sr_star = np.sqrt(sr_base ** 2 + r2_zero) / np.sqrt(1 - r2_zero)
    return sr_star - sr_base, sr_star

def r2_zero(y_true, y_pred):
    """
    Calculate zero-based R² (baseline is 0)
    y_true: array of true values (N,)
    y_pred: array of predicted values (N,)
    """
    rss = np.sum((y_true - y_pred)**2)
    tss = np.sum(y_true**2)
    return 1 - rss / tss

def calc_ic_daily(df, method='spearman'):
    """
    Calculate daily cross-sectional RankIC
    df: must contain ['signal_date','y_true','y_pred']
    """
    ics = (df.groupby('signal_date')
             .apply(lambda g: g['y_pred'].corr(g['y_true'], method=method))
             .dropna())
    mean_ic = ics.mean()
    std_ic  = ics.std(ddof=1)
    t_ic    = mean_ic / (std_ic / np.sqrt(len(ics))) if std_ic > 0 else np.nan
    pos_ratio = (ics > 0).mean()
    return mean_ic, t_ic, pos_ratio, ics

def calc_directional_metrics(y_true, y_pred, permnos=None):
    """
    - Sample-level sign prediction
    - If grouped by stock, calculate Overall, Up, Down for each stock and then average
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    if permnos is None:
        s_true = np.sign(y_true)
        s_pred = np.sign(y_pred)
        mask = s_true != 0
        s_true = s_true[mask]
        s_pred = s_pred[mask]

        overall_acc = np.mean(s_true == s_pred)

        up_mask = s_true > 0
        down_mask = s_true < 0
        up_acc = np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else 0
        down_acc = np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else 0

    else:
        df = pd.DataFrame({"permno": permnos, "yt": y_true, "yp": y_pred})
        overall_accs = []
        up_accs = []
        down_accs = []

        for _, g in df.groupby("permno"):
            s_true = np.sign(g["yt"].values)
            s_pred = np.sign(g["yp"].values)
            mask = s_true != 0
            s_true = s_true[mask]
            s_pred = s_pred[mask]
            if len(s_true) == 0:
                continue
            overall_accs.append(np.mean(s_true == s_pred))

            up_mask = s_true > 0
            down_mask = s_true < 0
            up_accs.append(np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else np.nan)
            down_accs.append(np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else np.nan)

        overall_acc = np.nanmean(overall_accs)
        up_acc = np.nanmean(up_accs)
        down_acc = np.nanmean(down_accs)

    return overall_acc, up_acc, down_acc

def regression_metrics(y_true, y_pred, k, meta=None, permnos=None):
    """
    Includes:
    - Regression metrics
    - Pointwise directional accuracy
    - Market cap group metrics
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)

    r2 = r2_zero(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    dir_acc, up_acc, down_acc = calc_directional_metrics(y_true, y_pred, permnos)

    metrics = {
        "R²_zero": r2,
        "RMSE": rmse,
        "MAE": mae,
        "MSE": mse,
        "Directional Accuracy": dir_acc,
        "Up_Directional_Acc": up_acc,
        "Down_Directional_Acc": down_acc
    }

    if meta is not None and "MKTCAP_PERCENTILE" in meta:
        top_mask = meta["MKTCAP_PERCENTILE"] >= 0.75
        bottom_mask = meta["MKTCAP_PERCENTILE"] <= 0.25

        if np.any(top_mask):
            yt_top = y_true[top_mask]
            yp_top = y_pred[top_mask]
            perm_top = permnos[top_mask] if permnos is not None else None
            r2_top = r2_zero(yt_top, yp_top)
            rmse_top = np.sqrt(mean_squared_error(yt_top, yp_top))
            mae_top = mean_absolute_error(yt_top, yp_top)
            mse_top = mean_squared_error(yt_top, yp_top)
            dir_top, up_top, down_top = calc_directional_metrics(yt_top, yp_top, perm_top)
            metrics.update({
                "Top25_R2_zero": r2_top,
                "Top25_RMSE": rmse_top,
                "Top25_MAE": mae_top,
                "Top25_MSE": mse_top,
                "Top25_Dir_Acc": dir_top,
                "Top25_Up_Acc": up_top,
                "Top25_Down_Acc": down_top
            })

        if np.any(bottom_mask):
            yt_bot = y_true[bottom_mask]
            yp_bot = y_pred[bottom_mask]
            perm_bot = permnos[bottom_mask] if permnos is not None else None
            r2_bot = r2_zero(yt_bot, yp_bot)
            rmse_bot = np.sqrt(mean_squared_error(yt_bot, yp_bot))
            mae_bot = mean_absolute_error(yt_bot, yp_bot)
            mse_bot = mean_squared_error(yt_bot, yp_bot)
            dir_bot, up_bot, down_bot = calc_directional_metrics(yt_bot, yp_bot, perm_bot)
            metrics.update({
                "Bottom25_R2_zero": r2_bot,
                "Bottom25_RMSE": rmse_bot,
                "Bottom25_MAE": mae_bot,
                "Bottom25_MSE": mse_bot,
                "Bottom25_Dir_Acc": dir_bot,
                "Bottom25_Up_Acc": up_bot,
                "Bottom25_Down_Acc": down_bot
            })

    return metrics

def f_statistic(y_true, y_pred, k):
    """Return F statistic and corresponding p-value"""
    n   = len(y_true)
    rss = np.sum((y_true - y_pred) ** 2)
    tss = np.sum(y_true ** 2)
    r2  = 1 - rss / tss
    if (r2 <= 0) or (n <= k):
        return 0.0, 1.0
    F = (r2 / k) / ((1 - r2) / (n - k))
    p = f_dist.sf(F, k, n - k)
    return F, p

def overall_interval_metrics_method1(y_all, yhat_all, k, permnos_all=None, meta_all=None):
    """
    Method 1: Calculate metrics for the entire interval at once (2016-2024, all samples concatenated)
    Returns: a dict, can be directly used for save_metrics()
    """
    base = regression_metrics(
        y_true=y_all,
        y_pred=yhat_all,
        k=k,
        meta=meta_all,
        permnos=permnos_all
    )
    F, p = f_statistic(y_all, yhat_all, k)
    base["F_stat"]     = F
    base["F_pvalue"]   = p
    base["N_obs"] = len(y_all)

    delta_cash, sr_star_cash = delta_sharpe(base["R²_zero"], sr_base=0)
    base["ΔSharpe_cash"]      = delta_cash
    base["Sharpe*_cash"]      = sr_star_cash

    delta_mkt , sr_star_mkt  = delta_sharpe(base["R²_zero"], sr_base=SR_MKT_EX)
    base["ΔSharpe_mkt"]       = delta_mkt
    base["Sharpe*_mkt"]       = sr_star_mkt

    return base

def sortino_ratio(rets, freq=252):
    """Calculate Sortino Ratio"""
    downside = rets[rets < 0]
    if len(downside) == 0:
        return np.inf
    mu = rets.mean() * freq
    sigma = np.sqrt((downside ** 2).mean()) * np.sqrt(freq)
    return mu / sigma

def cvar(rets, alpha=0.95):
    """Calculate CVaR"""
    q = np.quantile(rets, 1 - alpha)
    return rets[rets <= q].mean()

def save_metrics(metrics_dict, name, window, path="portfolio_metrics.csv"):
    """Save metrics to CSV file"""
    row = {'Model': name, 'Window': window}
    row.update(metrics_dict)

    if os.path.exists(path):
        df = pd.read_csv(path)
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    else:
        df = pd.DataFrame([row])

    df.to_csv(path, index=False)
    print(f"Metrics saved for {name}_w{window} to {path}")


/tmp/ipython-input-486884232.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")["Close"]
[*********************100%***********************]  1 of 1 completed

[INFO] S&P500 Excess Sharpe (2016–24) = 0.652



/tmp/ipython-input-486884232.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  rf_align = rf_series.reindex(sp_ret.index).fillna(method="ffill")


In [14]:
# Portfolio core class
# Transaction cost settings
TC_GRID = [0.0005, 0.001, 0.002, 0.003, 0.004]  # 5, 10, 20, 30, 40 bps
TC_TAG  = {
    0.0005: "tc5",
    0.001:  "tc10",
    0.002:  "tc20",
    0.003:  "tc30",
    0.004:  "tc40"
}

class PortfolioBacktester:
    def __init__(self):
        self.results = {}

    def calc_turnover(self, w_t, r_t, w_tp1):
        """Calculate turnover using the standard formula"""
        if w_t is None:
            return np.sum(np.abs(w_tp1))

        gross_ret = np.sum(w_t * r_t)
        if abs(1 + gross_ret) < 1e-8:
            return np.sum(np.abs(w_tp1))

        passive_weight = w_t * (1 + r_t) / (1 + gross_ret)
        turnover = np.sum(np.abs(w_tp1 - passive_weight))
        return turnover

    def create_portfolios_with_permno_tracking(self, signals, market_caps, permnos, top_pct=0.1, bottom_pct=0.1, weight_scheme="VW"):
        """
        Create portfolio weights based on signals, strictly tracking permno alignment.
        weight_scheme: 'VW' for value-weighted, 'EW' for equal-weighted
        """
        n_stocks = len(signals)
        top_n    = max(1, int(round(n_stocks * top_pct)))
        bottom_n = max(1, int(round(n_stocks * bottom_pct)))

        sorted_idx = np.argsort(signals)[::-1]

        top_idx = sorted_idx[:top_n]
        bottom_idx = sorted_idx[-bottom_n:]

        portfolio_data = {}

        # Long-only portfolio (Top 10%)
        long_weights = np.zeros(n_stocks)
        if len(top_idx) > 0:
            if weight_scheme == "VW":
                top_market_caps = market_caps[top_idx]
                if np.sum(top_market_caps) > 0:
                    long_weights[top_idx] = top_market_caps / np.sum(top_market_caps)
            else:
                long_weights[top_idx] = 1.0 / len(top_idx)

        portfolio_data['long_only'] = {
            'weights': long_weights,
            'permnos': permnos.copy(),
            'selected_permnos': permnos[top_idx] if len(top_idx) > 0 else np.array([])
        }

        # Short-only portfolio (Bottom 10%)
        short_weights = np.zeros(n_stocks)
        if len(bottom_idx) > 0:
            if weight_scheme == "VW":
                bottom_market_caps = market_caps[bottom_idx]
                if np.sum(bottom_market_caps) > 0:
                    short_weights[bottom_idx] = -bottom_market_caps / np.sum(bottom_market_caps)
            else:
                short_weights[bottom_idx] = -1.0 / len(bottom_idx)

        portfolio_data['short_only'] = {
            'weights': short_weights,
            'permnos': permnos.copy(),
            'selected_permnos': permnos[bottom_idx] if len(bottom_idx) > 0 else np.array([])
        }

        # Long-Short portfolio (Top long + Bottom short)
        ls_raw = long_weights + short_weights

        gross_target = 2.0
        current_gross = np.sum(np.abs(long_weights)) + np.sum(np.abs(short_weights))
        scale = gross_target / current_gross if current_gross > 1e-8 else 0.0
        ls_weights = scale * ls_raw

        ls_selected_permnos = np.concatenate([
            permnos[top_idx] if len(top_idx) > 0 else np.array([]),
            permnos[bottom_idx] if len(bottom_idx) > 0 else np.array([])
        ])

        portfolio_data['long_short'] = {
            'weights': ls_weights,
            'permnos': permnos.copy(),
            'selected_permnos': ls_selected_permnos
        }

        return portfolio_data

    def calculate_aligned_portfolio_return(self, portfolio_weights, portfolio_permnos, actual_returns, actual_permnos):
        """Calculate portfolio return strictly aligned by permno"""
        aligned_returns = np.zeros(len(portfolio_permnos))

        return_dict = dict(zip(actual_permnos, actual_returns))

        for i, permno in enumerate(portfolio_permnos):
            if permno in return_dict:
                aligned_returns[i] = return_dict[permno]

        portfolio_return = np.sum(portfolio_weights * aligned_returns)
        return portfolio_return, aligned_returns

    def calculate_metrics(self, returns, turnover_series=None):
        """Calculate portfolio metrics - only returns summary metrics, not full series"""
        returns = np.array(returns)

        annual_return = np.mean(returns) * 252
        annual_vol = np.std(returns, ddof=1) * np.sqrt(252)
        sharpe = annual_return / annual_vol if annual_vol > 0 else 0

        log_cum = np.cumsum(np.log1p(returns))
        peak_log = np.maximum.accumulate(log_cum)
        dd_log = peak_log - log_cum
        max_drawdown = 1 - np.exp(-dd_log.max())
        max_1d_loss = np.min(returns)

        avg_turnover = np.mean(turnover_series) if turnover_series is not None else 0

        sortino = sortino_ratio(returns)
        cvar95  = cvar(returns, alpha=0.95)

        result = {
            'annual_return': annual_return,
            'annual_vol': annual_vol,
            'sharpe': sharpe,
            'max_drawdown': max_drawdown,
            'max_1d_loss': max_1d_loss,
            'avg_turnover': avg_turnover,
            'sortino': sortino,
            'cvar95': cvar95
        }

        return result


In [15]:
# Chronos zero-shot prediction functions

def chronos_predict_batch(pipeline, X_batch, prediction_length=1, num_samples=10):
    """
    Batch prediction using Chronos.
    Args:
        pipeline: Chronos pipeline
        X_batch: input data (batch_size, sequence_length)
        prediction_length: prediction length
        num_samples: number of samples
    Returns:
        predictions: prediction results (batch_size,)
    """
    context_list = [torch.tensor(X_batch[i], dtype=torch.float32) for i in range(len(X_batch))]

    forecasts = pipeline.predict(
        context=context_list,
        prediction_length=prediction_length,
        num_samples=num_samples
    )

    predictions = np.array([forecast[0].mean() for forecast in forecasts])

    return predictions

def chronos_rolling_prediction(pipeline, X_data, batch_size=256, prediction_length=1):
    """
    Rolling window prediction using Chronos.
    Args:
        pipeline: Chronos model
        X_data: input features (n_samples, window_size)
        batch_size: batch size
        prediction_length: prediction length
    Returns:
        predictions: prediction results (n_samples,)
    """
    n_samples = len(X_data)
    predictions = np.zeros(n_samples)

    print(f"Running Chronos prediction on {n_samples} samples with batch size {batch_size}")

    for i in tqdm(range(0, n_samples, batch_size), desc="Chronos Prediction"):
        end_idx = min(i + batch_size, n_samples)
        batch_X = X_data[i:end_idx]

        batch_predictions = chronos_predict_batch(
            pipeline, batch_X, prediction_length=prediction_length
        )

        predictions[i:end_idx] = batch_predictions

    return predictions

print("Chronos prediction functions defined successfully")


Chronos prediction functions defined successfully


In [16]:
# ========== Main function for daily prediction and next-day rebalancing portfolio simulation ==========

def run_chronos_portfolio_backtest(start_year=2016, end_year=2024, window_sizes=None, model_names=None,
                                   npz_path="/content/drive/MyDrive/ERP Data/all_window_datasets_unscaled.npz"):
    """
Portfolio simulation (daily prediction and next-day rebalancing):
    1. Use Chronos T5 Base model for zero-shot prediction
    2. Daily prediction to daily signal
    3. Daily portfolio construction (T+1 rebalancing, strict permno alignment)
    4. Separate summary metrics and time series data
    """
    if window_sizes is None:
        window_sizes = [5, 21, 252, 512]
    if model_names is None:
        model_names = ["chronos base"]

    print("Starting daily rebalance portfolio backtesting simulation")

    backtester = PortfolioBacktester()

    datasets = load_datasets(npz_path)

    summary_results = []
    daily_series_data = []
    pred_rows = []

    WEIGHT_SCHEMES = ["VW", "EW"]

    for window in window_sizes:
        print(f"Processing window size: {window}")

        X_test = datasets[f"X_test_{window}"]
        y_test = datasets[f"y_test_{window}"]
        meta_test_dict = datasets[f"meta_test_{window}"].item()
        meta_test = pd.DataFrame.from_dict(meta_test_dict)

        permnos_test = meta_test["PERMNO"].values
        meta_test["signal_date"]  = pd.to_datetime(meta_test["date"])
        meta_test["ret_date"]     = pd.to_datetime(meta_test["ret_date"])
        market_caps = meta_test.get("MKTCAP", np.ones(len(permnos_test)))

        meta_test['date'] = pd.to_datetime(meta_test["date"])
        dates_test = meta_test['signal_date']

        for model_name in model_names:
            for scheme in WEIGHT_SCHEMES:
                all_y_true   = []
                all_y_pred   = []
                all_permnos  = []
                all_meta     = []
                print(f"  Model: {model_name}, Scheme: {scheme}")

                portfolio_daily_data = {
                    'long_only': {'returns': [], 'turnovers': [], 'dates': []},
                    'short_only': {'returns': [], 'turnovers': [], 'dates': []},
                    'long_short': {'returns': [], 'turnovers': [], 'dates': []}
                }

                prev_portfolio_data = {'long_only': None, 'short_only': None, 'long_short': None}

                signals_buf = {}

                for year in range(start_year, min(end_year + 1, 2025)):
                    print(f"  Processing year: {year}")

                    year_mask = (dates_test.dt.year == year)
                    if not np.any(year_mask):
                        continue

                    X_year = X_test[year_mask]
                    y_year = y_test[year_mask]
                    permnos_year = permnos_test[year_mask]
                    market_caps_year = market_caps[year_mask]
                    dates_year = dates_test[year_mask]
                    ret_dates_year = meta_test.loc[year_mask, 'ret_date'].values

                    batch_size = get_batch_size(window)
                    predictions_year = chronos_rolling_prediction(
                        pipeline=pipeline,
                        X_data=X_year,
                        batch_size=batch_size,
                        prediction_length=1
                    )

                    df_quarter = pd.DataFrame({
                        'signal_date': dates_year,
                        'ret_date': ret_dates_year,
                        'permno': permnos_year,
                        'market_cap': market_caps_year,
                        'actual_return': y_year,
                        'prediction': predictions_year
                    })

                    if scheme == 'VW':
                        df_q_save = df_quarter[['signal_date','ret_date','permno',
                                                'actual_return','prediction','market_cap']].copy()
                        df_q_save.rename(columns={'actual_return':'y_true',
                                                  'prediction':'y_pred'}, inplace=True)
                        df_q_save['model']  = model_name
                        df_q_save['window'] = window
                        pred_rows.append(df_q_save)

                    all_y_true.append(df_quarter['actual_return'].values)
                    all_y_pred.append(df_quarter['prediction'].values)
                    all_permnos.append(df_quarter['permno'].values)
                    all_meta.append(meta_test.loc[year_mask, :])

                    for signal_date, sig_grp in df_quarter.groupby('signal_date'):

                        daily_signals = (
                            sig_grp.groupby('permno')['prediction'].mean()
                                  .to_frame('prediction')
                                  .join(sig_grp.groupby('permno')['market_cap'].mean())
                        )
                        signals_buf[signal_date] = daily_signals

                        prev_date = signal_date - pd.tseries.offsets.BDay(1)
                        if prev_date not in signals_buf:
                            continue

                        sigs = signals_buf.pop(prev_date)

                        ret_grp = df_quarter[df_quarter['ret_date'] == signal_date]
                        if len(ret_grp) == 0:
                            continue

                        daily_actual_returns = (
                            ret_grp.groupby('permno')['actual_return']
                                   .mean()
                                   .reindex(sigs.index, fill_value=0)
                                   .values
                        )
                        daily_permnos = sigs.index.values

                        portfolios_data = backtester.create_portfolios_with_permno_tracking(
                            signals      = sigs['prediction'].values,
                            market_caps  = sigs['market_cap'].values,
                            permnos      = daily_permnos,
                            weight_scheme= scheme
                        )

                        for portfolio_type in ['long_only', 'short_only', 'long_short']:
                            portfolio_info = portfolios_data[portfolio_type]

                            portfolio_return, aligned_returns = backtester.calculate_aligned_portfolio_return(
                                portfolio_weights=portfolio_info['weights'],
                                portfolio_permnos=portfolio_info['permnos'],
                                actual_returns=daily_actual_returns,
                                actual_permnos=daily_permnos
                            )

                            if prev_portfolio_data[portfolio_type] is not None:
                                prev_w_ser = pd.Series(
                                    prev_portfolio_data[portfolio_type]['weights'],
                                    index=prev_portfolio_data[portfolio_type]['permnos']
                                )
                                cur_w_ser = pd.Series(
                                    portfolio_info['weights'],
                                    index=portfolio_info['permnos']
                                )

                                prev_r_ser = pd.Series(
                                    prev_portfolio_data[portfolio_type]['aligned_returns'],
                                    index=prev_portfolio_data[portfolio_type]['permnos']
                                )

                                aligned_prev_w = prev_w_ser.reindex(cur_w_ser.index, fill_value=0).values
                                aligned_prev_r = prev_r_ser.reindex(cur_w_ser.index, fill_value=0).values

                                aligned_cur_w = cur_w_ser.values

                                turnover = backtester.calc_turnover(
                                    w_t  = aligned_prev_w,
                                    r_t  = aligned_prev_r,
                                    w_tp1= aligned_cur_w
                                )
                            else:
                                turnover = np.sum(np.abs(portfolio_info['weights']))

                            portfolio_daily_data[portfolio_type]['returns'].append(portfolio_return)
                            portfolio_daily_data[portfolio_type]['turnovers'].append(turnover)
                            portfolio_daily_data[portfolio_type]['dates'].append(signal_date)

                            prev_portfolio_data[portfolio_type] = {
                                'weights'        : portfolio_info['weights'],
                                'permnos'        : portfolio_info['permnos'],
                                'aligned_returns': aligned_returns
                            }

                for portfolio_type in ['long_only', 'short_only', 'long_short']:
                    portfolio_data = portfolio_daily_data[portfolio_type]

                    if len(portfolio_data['returns']) > 0:
                        metrics = backtester.calculate_metrics(
                            returns=portfolio_data['returns'],
                            turnover_series=portfolio_data['turnovers']
                        )

                        rets = np.array(portfolio_data['returns'])
                        tovs = np.array(portfolio_data['turnovers'])

                        for tc in TC_GRID:
                            tag = TC_TAG[tc]
                            adj = rets - tovs * tc

                            ann_ret = adj.mean() * 252
                            ann_vol = adj.std(ddof=1) * np.sqrt(252)
                            sharpe  = ann_ret / ann_vol if ann_vol > 0 else 0

                            cum_adj = np.cumprod(1 + adj)
                            mdd = ((cum_adj - np.maximum.accumulate(cum_adj)) /
                                   np.maximum.accumulate(cum_adj)).min()

                            metrics[f'{tag}_annual_return'] = ann_ret
                            metrics[f'{tag}_annual_vol']    = ann_vol
                            metrics[f'{tag}_sharpe']        = sharpe
                            metrics[f'{tag}_max_drawdown']  = mdd

                        summary_results.append({
                            'scheme': scheme,
                            'model': model_name,
                            'window': window,
                            'portfolio_type': portfolio_type,
                            **metrics
                        })

                        rets_arr = np.array(portfolio_data['returns'])
                        tovs_arr = np.array(portfolio_data['turnovers'])
                        cum_no_tc = np.log1p(rets_arr).cumsum()

                        tc_ret_dict = {}
                        tc_cum_dict = {}
                        for tc in TC_GRID:
                            tag = TC_TAG[tc]
                            r = rets_arr - tovs_arr * tc
                            tc_ret_dict[tag] = r
                            tc_cum_dict[tag] = np.log1p(r).cumsum()

                        for i, date in enumerate(portfolio_data['dates']):
                            row = {
                                'scheme'        : scheme,
                                'model'         : model_name,
                                'window'        : window,
                                'portfolio_type': portfolio_type,
                                'date'          : str(date),
                                'return'        : rets_arr[i],
                                'turnover'      : tovs_arr[i],
                                'cumulative'    : cum_no_tc[i],
                            }
                            for tag in TC_TAG.values():
                                row[f'{tag}_return']     = tc_ret_dict[tag][i]
                                row[f'{tag}_cumulative'] = tc_cum_dict[tag][i]

                            daily_series_data.append(row)

                if scheme == "VW" and len(all_y_true) > 0:
                    y_all    = np.concatenate(all_y_true)
                    yhat_all = np.concatenate(all_y_pred)
                    perm_all = np.concatenate(all_permnos)
                    meta_all = pd.concat(all_meta, ignore_index=True)

                    k = X_test.shape[1]

                    m1_metrics = overall_interval_metrics_method1(
                        y_all, yhat_all, k,
                        permnos_all=perm_all,
                        meta_all=meta_all
                    )

                    full_pred_df = pd.concat(pred_rows, ignore_index=True)
                    mean_ic, t_ic, pos_ic, _ = calc_ic_daily(full_pred_df, method='spearman')
                    m1_metrics['RankIC_mean']  = mean_ic
                    m1_metrics['RankIC_t']     = t_ic
                    m1_metrics['RankIC_pos%']  = pos_ic

                    save_metrics(m1_metrics, name=model_name, window=window,
                        path="/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_metrics.csv")

    summary_df = pd.DataFrame(summary_results)
    daily_df = pd.DataFrame(daily_series_data) if daily_series_data else pd.DataFrame()

    tc_columns = [c for c in summary_df.columns if c.startswith('tc')]
    summary_df[tc_columns] = summary_df[tc_columns].fillna(0.0)

    def save_split_by_scheme(df, base_filename):
        """Helper function to save files split by scheme"""
        if df.empty:
            print(f"Warning: DataFrame is empty, skipping save for {base_filename}")
            return None, None

        vw_df = df[df['scheme'] == 'VW']
        ew_df = df[df['scheme'] == 'EW']

        vw_filename = f"/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/{base_filename}_VW.csv"
        ew_filename = f"/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/{base_filename}_EW.csv"

        vw_df.to_csv(vw_filename, index=False)
        ew_df.to_csv(ew_filename, index=False)

        print(f"VW results saved to {vw_filename}")
        print(f"EW results saved to {ew_filename}")

        return vw_filename, ew_filename

    save_split_by_scheme(summary_df, "portfolio_results_daily_rebalance")

    if not daily_df.empty:
        save_split_by_scheme(daily_df, "portfolio_daily_series")

    if pred_rows:
        pred_df = pd.concat(pred_rows, ignore_index=True)
        pred_df.to_csv("/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/predictions_daily.csv", index=False)
        print(f"Saved {len(pred_df)} prediction rows to predictions_daily.csv")

    print(f"Generated {len(summary_results)} portfolio summary records")
    print(f"Generated {len(daily_series_data)} daily series records")

    return summary_df, daily_df, backtester


In [17]:
# ========== Results Saving ==========

import os

results_dir = "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results"
figures_dir = "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_figures"

os.makedirs(results_dir, exist_ok=True)
os.makedirs(figures_dir, exist_ok=True)

In [18]:
# ========== Run Backtest ==========

print("Starting Chronos T5-Base Portfolio Backtesting...")

summary_results, daily_series, backtester = run_chronos_portfolio_backtest(
    start_year=START_YEAR,
    end_year=END_YEAR,
    window_sizes=WINDOW_SIZES
)

print("\n" + "="*60)
print("CHRONOS T5-BASE PORTFOLIO BACKTESTING RESULTS")
print("="*60)

print("\nSummary Results:")
print(summary_results.round(4))


Starting Chronos T5-Base Portfolio Backtesting...
Starting Daily Rebalance Portfolio Backtesting Simulation
Data loaded successfully! Keys: ['X_train_5', 'y_train_5', 'meta_train_5', 'market_caps_train_5', 'X_test_5', 'y_test_5', 'meta_test_5', 'market_caps_test_5', 'X_train_21', 'y_train_21', 'meta_train_21', 'market_caps_train_21', 'X_test_21', 'y_test_21', 'meta_test_21', 'market_caps_test_21', 'X_train_252', 'y_train_252', 'meta_train_252', 'market_caps_train_252', 'X_test_252', 'y_test_252', 'meta_test_252', 'market_caps_test_252', 'X_train_512', 'y_train_512', 'meta_train_512', 'market_caps_train_512', 'X_test_512', 'y_test_512', 'meta_test_512', 'market_caps_test_512']
Available datasets: 32 files
Processing window size: 5
  Model: chronos base, Scheme: VW
  Processing year: 2016
Running Chronos prediction on 12475 samples with batch size 1024


Chronos Prediction: 100%|██████████| 13/13 [00:08<00:00,  1.59it/s]


  Processing year: 2017
Running Chronos prediction on 12434 samples with batch size 1024


Chronos Prediction: 100%|██████████| 13/13 [00:07<00:00,  1.70it/s]


  Processing year: 2018
Running Chronos prediction on 12326 samples with batch size 1024


...

  Processing year: 2023
Running Chronos prediction on 12367 samples with batch size 16


Chronos Prediction: 100%|██████████| 773/773 [07:17<00:00,  1.77it/s]


  Processing year: 2024
Running Chronos prediction on 12394 samples with batch size 16


Chronos Prediction: 100%|██████████| 775/775 [07:18<00:00,  1.77it/s]


VW results saved to /content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_results_daily_rebalance_VW.csv
EW results saved to /content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_results_daily_rebalance_EW.csv
VW results saved to /content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_VW.csv
EW results saved to /content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_EW.csv
Saved 443400 prediction rows to predictions_daily.csv
Generated 24 portfolio summary records
Generated 52272 daily series records
\n============================================================
CHRONOS T5-BASE PORTFOLIO BACKTESTING RESULTS
\nSummary Results:
   scheme         model  window portfolio_type  annual_return  annual_vol  \
0      VW  chronos base       5      long_only         0.1252      0.1982   
1      VW  chronos base       5     short_only        -0.1585      0.2050 

In [19]:
# ---------- Main function for 5-factor regression -----------
def run_factor_regression(port_ret, factors, use_excess=True):
    df = pd.concat([port_ret, factors], axis=1, join='inner').dropna()
    df.columns = ['ret'] + list(factors.columns)

    if use_excess:
        y = df['ret'].values
    else:
        y = df['ret'].values - df['rf'].values

    X = df[['mktrf','smb','hml','rmw','cma','umd']].values
    X = sm.add_constant(X)

    model = sm.OLS(y, X)
    res = model.fit()
    alpha = res.params[0]          
    resid_std = res.resid.std(ddof=1)

    ir_daily = alpha / resid_std          
    ir_annual = ir_daily * np.sqrt(252)   

    y_hat = np.asarray(res.fittedvalues)  

    out = {
        'N_obs'            : len(y),
        'alpha_daily'      : alpha,
        'alpha_annual'     : alpha*252,
        't_alpha'          : res.tvalues[0],
        'IR_daily'         : ir_daily,
        'IR_annual'        : ir_annual,
        'R2_zero'          : r2_zero(y, y_hat),
    }

    factor_names = ['MKT','SMB','HML','RMW','CMA','UMD']
    for i, fac in enumerate(factor_names, start=1):
        out[f'beta_{fac}'] = res.params[i]
        out[f't_{fac}']    = res.tvalues[i]

    return out

# ---------- Batch run (EW/VW, three portfolio types) ----------
def batch_factor_analysis(
    daily_df: pd.DataFrame,
    factors_path: str,
    scheme: str,
    tc_levels=(0, 5, 10, 20, 40),
    portfolio_types=('long_only','short_only','long_short'),
    model_filter=None,
    window_filter=None,
    gross_only=False,            
    out_dir='/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/factor_IR_results',
):
    """
    Generate a CSV containing IR results.
    gross_only=True  → only tc=0; False → all tc_levels.
    """
    import os
    os.makedirs(out_dir, exist_ok=True)

    fac = (pd.read_csv(factors_path, parse_dates=['date'])
             .set_index('date')
             .sort_index())

    sub = daily_df[daily_df['scheme'] == scheme].copy()
    if model_filter is not None:
        sub = sub[sub['model'].isin(model_filter)]
    if window_filter is not None:
        sub = sub[sub['window'].isin(window_filter)]

    tc_iter = (0,) if gross_only else tc_levels
    results = []

    for (model, win, ptype), g in sub.groupby(['model','window','portfolio_type']):
        g = g.sort_values('date').set_index(pd.to_datetime(g['date']))

        for tc in tc_iter:
            col = 'return' if tc == 0 else f'tc{tc}_return'
            if col not in g.columns:
                continue  
            port_ret = g[col]
            stats = run_factor_regression(port_ret, fac, use_excess=True)
            stats.update({
                'scheme'        : scheme,
                'model'         : model,
                'window'        : win,
                'portfolio_type': ptype,
                'tc_bps'        : tc,
            })
            results.append(stats)

    df_out = pd.DataFrame(results)[[
        'scheme','model','window','portfolio_type','tc_bps','N_obs',
        'alpha_daily','alpha_annual','t_alpha',
        'IR_daily','IR_annual','R2_zero',
        'beta_MKT','t_MKT','beta_SMB','t_SMB',
        'beta_HML','t_HML','beta_RMW','t_RMW',
        'beta_CMA','t_CMA','beta_UMD','t_UMD'
    ]]

    tag = 'gross' if gross_only else 'net'
    fname = f'5_factor_analysis_{scheme}_{tag}.csv'
    df_out.to_csv(os.path.join(out_dir, fname), index=False)
    print(f'[Saved] {fname} ')
    return df_out



def run_all_factor_tests(vw_csv="/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_VW.csv",
                         ew_csv="/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_EW.csv",
                         factor_csv="/content/drive/MyDrive/ERP Data/5_Factors_Plus_Momentum.csv",
                         save_dir="/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results",
                         y_is_excess=True,
                         hac_lags=5,
                         save_txt=True):
    vw_df = pd.read_csv(vw_csv)
    ew_df = pd.read_csv(ew_csv)

    vw_gross = batch_factor_analysis(
        vw_df, factor_csv, scheme='VW', gross_only=True)    
    vw_net   = batch_factor_analysis(
        vw_df, factor_csv, scheme='VW', gross_only=False)   

    ew_gross = batch_factor_analysis(
        ew_df, factor_csv, scheme='EW', gross_only=True)
    ew_net   = batch_factor_analysis(
        ew_df, factor_csv, scheme='EW', gross_only=False)

    return vw_gross, vw_net, ew_gross, ew_net


vw_gross, vw_net, ew_gross, ew_net = run_all_factor_tests()

[Saved] 5_factor_analysis_VW_gross.csv 
[Saved] 5_factor_analysis_VW_net.csv 
[Saved] 5_factor_analysis_EW_gross.csv 
[Saved] 5_factor_analysis_EW_net.csv 


In [20]:
# === File paths ===
rf_file = "/content/drive/MyDrive/ERP Data/CRSP_2016_2024_top50_with_exret.csv"
vw_file = "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_VW.csv"
ew_file = "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_EW.csv"

# === Load risk-free rate (rf) data ===

rf_df = pd.read_csv(rf_file, usecols=["date", "rf"])
rf_df["date"] = pd.to_datetime(rf_df["date"])
rf_dict = dict(zip(rf_df["date"], rf_df["rf"]))


def adjust_returns_with_rf_grouped(file_path, output_path):
    df = pd.read_csv(file_path)
    # Parse date with flexible format
    df["date"] = pd.to_datetime(df["date"], format='mixed', dayfirst=True)

    # Find all return columns (exclude cumulative)
    return_cols = [col for col in df.columns if "return" in col and "cumul" not in col]

    # Set portfolio_type order to avoid groupby sorting issues
    order = ["long_only", "short_only", "long_short"]
    df["portfolio_type"] = pd.Categorical(df["portfolio_type"], categories=order, ordered=True)

    df_list = []
    for _, group in df.groupby(["scheme", "model", "window", "portfolio_type"], sort=False):
        group = group.sort_values("date").copy()
        for col in return_cols:
            group[col] = group.apply(lambda row: row[col] + rf_dict.get(row["date"], 0), axis=1)
            cum_col = col.replace("return", "cumulative")
            group[cum_col] = np.log1p(group[col]).cumsum()
        df_list.append(group)

    df_new = pd.concat(df_list).sort_values(["scheme", "model", "window", "portfolio_type", "date"])
    df_new.to_csv(output_path, index=False)
    print(f"Finished: {output_path}")

adjust_returns_with_rf_grouped(vw_file, "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_VW_with_rf.csv")
adjust_returns_with_rf_grouped(ew_file, "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_EW_with_rf.csv")


/tmp/ipython-input-3268058695.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, group in df.groupby(["scheme", "model", "window", "portfolio_type"], sort=False):


Finish: /content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_VW_with_rf.csv


/tmp/ipython-input-3268058695.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, group in df.groupby(["scheme", "model", "window", "portfolio_type"], sort=False):


Finish: /content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_EW_with_rf.csv


In [21]:
# Download S&P500 (2016-2024)
sp500 = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")
price_col = "Adj Close" if "Adj Close" in sp500.columns else "Close"
sp500["daily_return"] = sp500[price_col].pct_change().fillna(0)
# Cumulative log return
sp500["cum_return"] = np.cumsum(np.log1p(sp500["daily_return"]))
sp500 = sp500[["cum_return"]]
sp500.index = pd.to_datetime(sp500.index)

# Configuration
files = [
    ("VW", "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_VW_with_rf.csv"),
    ("EW", "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_daily_series_EW_with_rf.csv")
]
tc_levels = [0, 5, 10, 20, 40]      # Transaction cost (bps)
windows = [5, 21, 252, 512]         # Window sizes
strategies = ["long_only", "short_only", "long_short"]

output_dir = "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_figures"
os.makedirs(output_dir, exist_ok=True)

# Economic event periods (for gray shading)
crisis_periods = [
    (datetime(2018, 6, 1), datetime(2019, 1, 1), "US-China Trade War"),
    (datetime(2020, 2, 1), datetime(2020, 7, 1), "COVID-19"),
    (datetime(2022, 2, 1), datetime(2022, 6, 1), "Russia-Ukraine War"),
    (datetime(2023, 1, 1), datetime(2023, 4, 1), "US Bank Crisis"),
]

def plot_comparison_styled(df, scheme, tc, window):
    plt.figure(figsize=(15, 12))
    model_names = df["model"].unique()
    colors = plt.cm.tab10(np.linspace(0, 1, len(model_names)))

    offset_step = 0.02

    for i, strat in enumerate(strategies, 1):
        ax = plt.subplot(3, 1, i)

        # Plot S&P500 baseline
        plt.plot(sp500.index, sp500["cum_return"],
                 color="black", lw=2.5, label="S&P500 (Total Return)", zorder=10)

        for idx, model_name in enumerate(model_names):
            sub = df[(df["window"] == window) &
                     (df["portfolio_type"] == strat) &
                     (df["model"] == model_name)].sort_values("date")
            if sub.empty:
                continue

            if tc == 0:
                ret_col = "return"
            else:
                ret_col = f"tc{tc}_return"

            if ret_col not in sub.columns:
                continue

            log_cum = np.cumsum(np.log1p(sub[ret_col].values))

            y_shift = idx * offset_step
            plt.plot(sub["date"], log_cum + y_shift,
                     label=f"{model_name} ({strat.replace('_',' ').title()})",
                     lw=2, color=colors[idx], alpha=0.9)

        for start, end, label in crisis_periods:
            ax.axvspan(start, end, color='grey', alpha=0.3)
            ax.text(start + pd.Timedelta(days=10),
                    ax.get_ylim()[1]*0.92, label, fontsize=8, color='grey')
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.set_ylabel("Cumulative log return (start = 0)")
        ax.set_title(f"{scheme} | Window={window} | Strategy={strat} | TC={tc} bps")
        ax.grid(alpha=0.3)
        plt.xticks(rotation=30)
        plt.legend(bbox_to_anchor=(1.04, 1), loc='upper left', fontsize=8)

    plt.tight_layout()
    fname = f"{scheme}_window{window}_TC{tc}.png"
    plt.savefig(os.path.join(output_dir, fname), dpi=300, bbox_inches='tight')
    plt.close()

# Main loop to generate all figures
for scheme, file_path in files:
    df = pd.read_csv(file_path)
    df["date"] = pd.to_datetime(df["date"])
    for tc in tc_levels:
        for window in windows:
            plot_comparison_styled(df, scheme, tc, window)

print(f"All figures have been generated and saved to: {output_dir}/")


/tmp/ipython-input-917073965.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500 = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")
[*********************100%***********************]  1 of 1 completed


All figures have been generated and saved to: /content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_figures/


In [22]:

# Load R²_zero from portfolio_metrics.csv
metrics_df = pd.read_csv("/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_metrics.csv")[["Model", "Window", "R²_zero"]]
metrics_df.rename(columns={"Model": "model", "Window": "window"}, inplace=True)

# Process VW/EW files
for fname in ["/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_results_daily_rebalance_VW.csv", "/content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_results_daily_rebalance_EW.csv"]:
    df = pd.read_csv(fname)

    # Merge R²_zero by model and window
    df = df.merge(metrics_df, on=["model", "window"], how="left")

    rows = []
    for _, row in df.iterrows():
        r2 = float(row["R²_zero"]) if not pd.isna(row["R²_zero"]) else 0.0
        if row["portfolio_type"] == "long_only":
            d_sr, sr_star = delta_sharpe(r2, SR_MKT_EX)
            row["ΔSharpe"]  = d_sr
            row["Sharpe*"]  = sr_star
            row["baseline"] = f"SPX_excess ({SR_MKT_EX:.2f})"
        else:
            d_sr, sr_star = delta_sharpe(r2, 0)
            row["ΔSharpe"]  = d_sr
            row["Sharpe*"]  = sr_star
            row["baseline"] = "cash (0)"
        rows.append(row)

    pd.DataFrame(rows).to_csv(fname, index=False)
    print(f"[Update] ΔSharpe has been written to {fname}")

[Update] ΔSharpe has been written to /content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_results_daily_rebalance_VW.csv
[Update] ΔSharpe has been written to /content/drive/MyDrive/chronos_t5_base_project_portfolio/chronos_results/portfolio_results_daily_rebalance_EW.csv
